In [1]:

import json
import numpy as np
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [2]:
def label (dataCategory,datas):
    labels = []
    if dataCategory == 'train':
        for idx in range(len(datas)):
            data = datas[idx]    
            author = np.array(data['authors'])
            prolific_author = author<100
            prolific_author = author[prolific_author]
            empty_author = np.zeros(101) # 100 plus 1 negative
            if len(prolific_author)!=0:
                empty_author[prolific_author]=1
            else:
                empty_author[100]=1
            labels.append(empty_author)
 
    return labels

In [3]:
def text_feature (dataCategory, datas, model):
    title = []
    abstract = []
    for data in datas:
        single_abstract = data['abstract']
        single_title = data['title']
        title.append(single_title)
        abstract.append(single_abstract)
    common_texts = title+abstract
    # doc2vec code from https://radimrehurek.com/gensim/models/doc2vec.html
    documents = [TaggedDocument(doc,[i])for i,doc in enumerate(common_texts)]
    if dataCategory == 'train':
        model=Doc2Vec(documents,vector_size = 256,window = 2,min_count = 1)#no worker
        model.train(documents, total_examples=model.corpus_count, epochs=30)# 20
    full_text = []
    for data in datas:
        abstract2 = data['abstract']
        title2 = data['title']
        sums = abstract2+title2
        text_empty = np.zeros(256)
        for word in sums:
            word_single = []
            word_single.append(str(word))
        text_empty+= model.infer_vector(word_single)   
        full_text.append(text_empty)
    full_text = np.array(full_text)
    return (full_text, model)

In [4]:
def onehot_feature (dataCategory,datas, co_idx={}):
    if dataCategory == 'train':
        author = {}
        co_idx = {}
        coauthor = []
        for record in datas:
            author_record = np.array(record['authors'])
            pa_bol = author_record<100
            ca_bol = author_record>=100
            co_author = author_record[ca_bol]
            if author_record[pa_bol == True].size != 0:
                for i in co_author:
                    author[i] = 1
        index = author.keys()
        coauthorList = list(index)
        
        for i in range(len(coauthorList)):
            co_idx[coauthorList[i]]=i

        for record in datas:
            author_record = np.array(record['authors'])
            pa_bol = author_record<100
            ca_bol = author_record>=100
            co_author = author_record[ca_bol]
            colen = len(coauthorList)
            co_empty = np.zeros(colen+1)
            if author_record[pa_bol == True].size == 0:
                co_empty[colen]=1.
            else:
                for j in co_author:
                    idx = co_idx[j]
                    co_empty[idx] = 1
            coauthor.append(co_empty)
        
        coauthor = np.array(coauthor)
        
        # year 
        year = []
        for record in datas:
            year_empty = np.zeros(20)
            year_record = record['year']
            year_empty[year_record] = 1
            year.append(year_empty)
        year = np.array(year)
        
        # venue    
        venue = []
        np.array(venue)
        for record in datas:
            venue_empty = np.zeros(466)# 465 +1 empty
            venue_data = record['venue']
            if venue_data == '':
                venue_data = 465
            venue_empty[venue_data] = 1
            venue.append( venue_empty)
        onehot = np.concatenate((venue,coauthor),axis =1 )
        return (onehot, co_idx)
    
    elif dataCategory == 'test':
        author = {}
        index = co_idx.keys()
        coauthorList = list(index)
        coauthor = []
        for record in datas:
            author_record = np.array(record['coauthors'])
            colen = len(coauthorList)
            co_empty = np.zeros(colen+1)
            for au in author_record:
                if au in coauthorList:
                    co_empty[co_idx[au]] = 1
                else:
                    co_empty[-1] = 1
            coauthor.append(co_empty)
        coauthor = np.array(coauthor)
        
        # year 
        year = []
        for record in datas:
            year_empty = np.zeros(20)
            year_record = record['year']
            year_empty[year_record] = 1
            year.append(year_empty)
        year = np.array(year)
        
        #venue
        venue = []
        np.array(venue)
        for record in datas:
            venue_empty = np.zeros(466)# 465 +1 empty
            venue_data = record['venue']
            if venue_data == '':
                venue_data = 465
            venue_empty[venue_data] = 1
            venue.append( venue_empty)
        onehot = np.concatenate((venue,coauthor),axis =1 )
        return (onehot, None)

In [5]:
def feature (dataCategory,datas, model, co_list):
    text, model = text_feature (dataCategory, datas, model)
    one_hot, co_list = onehot_feature(dataCategory,datas, co_list)
    feature = np.concatenate((text,one_hot),axis =1 )
    return (feature, model, co_list)
    

In [6]:
train_data = json.load(open('train.json'))
test_data = json.load(open('test.json'))

In [7]:
model = None
co_idx = None
train_feature, model, co_idx = feature('train',train_data, model, co_idx)
np.save('feature_train.npy',train_feature)

In [8]:
test_feature, _, _ = feature('test',test_data, model, co_idx)
np.save('feature_test.npy',test_feature)

In [9]:
label_train = label ('train',train_data)
np.save('labels.npy',label_train)

In [10]:
len(train_feature)
    

25793

In [11]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [12]:
import json
import numpy as np
from typing import *
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch
import tensorflow as tf

import random
import numpy as np
import pandas as pd
import math
from collections import Counter
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

In [243]:
train_small = train_feature[0:4000]
label_small = label_train[0:4000]

In [244]:
x_train,x_dev,y_train,y_dev=train_test_split(train_small,label_small,test_size=0.20,random_state=48)

In [245]:
x_train.shape

(3200, 7298)

In [246]:

lab_train= np.vstack(y_train)
lab_train.shape

(3200, 101)

In [247]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier

In [248]:
import warnings
warnings.filterwarnings("ignore")

In [267]:
for clf in [DecisionTreeClassifier]:
    model = OneVsRestClassifier(clf()) 
    model.fit(x_train,lab_train)
    print(clf.__name__,'train score：%.2f' % model.score(x_train,lab_train), 'test score：%.2f' % model.score(x_dev, y_dev))
    print(clf.__name__,"f1_score",f1_score(lab_train,model.predict(x_train),average="samples"))
    out_list = model.predict(test_feature)
    #print(model.predict(test_feature))

DecisionTreeClassifier train score：0.99 test score：0.79
DecisionTreeClassifier f1_score 0.988125


In [288]:
for clf in [LogisticRegression]:
    model = OneVsRestClassifier(clf()) 
    model.fit(x_train,lab_train)
    print(clf.__name__,'train score：%.2f' % model.score(x_train,lab_train), 'test score：%.2f' % model.score(x_dev, y_dev))
    print(clf.__name__,"f1_score",f1_score(lab_train,model.predict(x_train),average="samples"))
    out_list = model.predict(test_feature)

LogisticRegression train score：0.72 test score：0.72
LogisticRegression f1_score 0.72390625


In [251]:
for clf in [SVC]:
    model = OneVsRestClassifier(clf()) 
    model.fit(x_train,lab_train)
    print(clf.__name__,'train score：%.2f' % model.score(x_train,lab_train), 'test score：%.2f' % model.score(x_dev, y_dev))
    print(clf.__name__,"f1_score",f1_score(lab_train,model.predict(x_train),average="samples"))
    out_list = model.predict(test_feature)

SVC train score：0.81 test score：0.71
SVC f1_score 0.8150232843137255


In [259]:
for clf in [MLPClassifier]: 
    model = OneVsRestClassifier(clf()) 
    model.fit(x_train,lab_train)
    print(clf.__name__,'train score：%.2f' % model.score(x_train,lab_train), 'test score：%.2f' % model.score(x_dev, y_dev))
    print(clf.__name__,"f1_score",f1_score(lab_train,model.predict(x_train),average="samples"))
    out_list = model.predict(test_feature)

MLPClassifier train score：0.98 test score：0.78
MLPClassifier f1_score 0.9828125


In [276]:
def transfer_f(list1):
    outputlist = []
    new_list = [] 
    for i in list1:
        for j in range(len(i)):
            if(i[j]==1)and(j!=100):
                new_list.append(j)
        outputlist.append((new_list)) 
        new_list = []
    return outputlist

In [277]:
final  = transfer_f(out_list)
for i in range(len(final)):
    if(len(final[i])==0):
        final[i]=[-1]

In [278]:
id_list = []
for i in range(len(test_data)):
    id_list.append(i)

In [285]:
import pandas as pd
df = pd.DataFrame({'ID':id_list,'Predict':final})

In [286]:
for i in range(len(final)):
    it = df.loc[i]["Predict"]
    if(it!=-1):
        if(len(it)==1):
            df.loc[i]["Predict"] = final

In [287]:
df['Predict'] = df['Predict'].apply(lambda x:','.join(str(i) for i in x))
df

,ID,Predict
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
795,795,-1
796,796,-1
797,797,-1
798,798,-1


In [282]:
df.to_csv(r'F:/LR2.csv')